In [13]:
import pandas as pd
dataset = [
    ['A', 'D', 5],
    ['A', 'B', 7],
    ['B', 'D', 9],
    ['B', 'C', 8],
    ['B', 'E', 7],
    ['C', 'E', 5],
    ['D', 'E', 15],
    ['D', 'F', 6],
    ['E', 'F', 8],
    ['E', 'G', 9],
    ['F', 'G', 11],
]


In [14]:
#pirmas
df = pd.DataFrame(dataset, columns=['start', 'end', 'distance'])
# print(df)

startingPoint = 'D'
chosenPoints = [startingPoint]
connections = []
while True:
    # print('---')
    foundNeighbours = df[(df["start"].isin(chosenPoints) & ~df["end"].isin(chosenPoints)) \
        | ~df["start"].isin(chosenPoints) & df["end"].isin(chosenPoints)] 
    if not len(foundNeighbours):
        break
    # print(foundNeighbours)
    closestNeighbour = foundNeighbours.loc[foundNeighbours.distance.idxmin()]
    
    # print(closestNeighbour)
    if not closestNeighbour.start in chosenPoints:
        chosenPoints.append(closestNeighbour.start)
        connections.append([closestNeighbour.end, closestNeighbour.start, closestNeighbour.distance])
    elif not closestNeighbour.end in chosenPoints:
        chosenPoints.append(closestNeighbour.end)
        connections.append([closestNeighbour.start, closestNeighbour.end, closestNeighbour.distance])


# print(chosenPoints)
chosenPoints.sort()
print(chosenPoints)
connections = pd.DataFrame(connections, columns=['start', 'end', 'distance'])
print(connections)
    


['A', 'B', 'C', 'D', 'E', 'F', 'G']
  start end  distance
0     D   A         5
1     D   F         6
2     A   B         7
3     B   E         7
4     E   C         5
5     E   G         9


In [15]:
#antras
clusterPoints = chosenPoints
clusterConnections = connections
numberOfClusters = 3
for i in range(numberOfClusters - 1):
    clusterConnections = clusterConnections.drop(clusterConnections["distance"].idxmax())

clusterConnections.index = range(len(clusterConnections.index))
# print(clusterConnections)

foundClusters = []
for index in range(len(clusterConnections)):
    connection = clusterConnections.loc[index]
    curr = (connection.start, connection.end)
    
    addedToCluster = False
    for cluster in foundClusters:
        if connection.start in cluster:
            cluster.append(connection.end)
            addedToCluster = True
            break
        elif connection.end in cluster:
            cluster.append(connection.start)
            addedToCluster = True
            break

    if not addedToCluster:
        foundClusters.append([connection.start, connection.end])
    print(foundClusters)

for point in clusterPoints:
    hasCluster = True
    for cluster in foundClusters:
        if not point in cluster:
            hasCluster = False
        if point in cluster:
            hasCluster = True
            break


    if not hasCluster:
        foundClusters.append([point])

print(foundClusters)

[['D', 'A']]
[['D', 'A', 'F']]
[['D', 'A', 'F'], ['B', 'E']]
[['D', 'A', 'F'], ['B', 'E', 'C']]
[['D', 'A', 'F'], ['B', 'E', 'C'], ['G']]


In [145]:
#trecias
dataset = [
    ["Vardas1", 1, 1, 1, 1, 1, 1, 1, 0, 0, 1],
    ["Vardas2", 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    ["Vardas3", 1, 0, 1, 0, 0, 0, 1, 0, 0, 1],
    ["Vardas4", 1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
    ["Vardas5", 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    ["Vardas6", 0, 1, 1, 1, 1, 0, 1, 0, 0, 0],
    ["Vardas7", 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
    ["Vardas8", 1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
    ["Vardas9", 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
    ["Vardas10", 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
    ["Vardas11", 1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
    ["Vardas12", 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
    ["Vardas13", 1, 0, 1, 1, 1, 1, 1, 0, 0, 1],
    ["Vardas14", 1, 1, 1, 1, 1, 0, 1, 0, 0, 1],
    ["Vardas15", 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
]

valueColumns = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10']
df = pd.DataFrame(dataset, columns=['pacientas', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10'])
allPoints = df.drop("pacientas", axis=1)

from scipy.spatial.distance import cdist
import numpy as np

radius = 2
error = 0.01

centers = pd.DataFrame([], columns=valueColumns)
points = allPoints
while len(points):
    # print(points)
    current_point = points.sample(n = 1)
    # current_point = points.loc[[4]]
    current_point_index = current_point.index.values[0]
    # print('index', current_point_index)
    # print('current point')
    # print(current_point)
    # print('----')

    distances = pd.DataFrame(cdist(points, points, 'euclidean'))
    distancesToPoint = distances[current_point_index]
    neighboursIndexes = [index for index, distance in enumerate(distancesToPoint) if distance < radius]
    neighbours = points.loc[neighboursIndexes]
    # print('neighbours')
    # print(neighbours)
    # print('----')

    center = [np.mean(neighbours[column].values) for column in neighbours]
    # print('center')
    # print(center)
    # print('----')


    # print(current_point - center)
    # print(np.linalg.norm(current_point - center))
    # print('--')
    # print('--')
    # print('--')
    # print('--')
    while np.linalg.norm(current_point - center) > error:
    # while True:
        print('----')
        print(center)
        current_point = pd.DataFrame([center], columns=valueColumns)
        print(current_point)
        pointsWithCenter = points.append(current_point, ignore_index = True)
        current_point_index = len(pointsWithCenter) - 1
        print(pointsWithCenter)
        
        distances = pd.DataFrame(cdist(pointsWithCenter, pointsWithCenter, 'euclidean'))
        distancesToPoint = distances[current_point_index]
        neighboursIndexes = [index for index, distance in enumerate(distancesToPoint) if distance < radius]
        neighbours = pointsWithCenter.loc[neighboursIndexes]
        # print('---')
        # print(neighbours)

        center = [np.mean(neighbours[column].values) for column in neighbours]
        points = points.drop(current_point_index)

        # break;
    # while np.linalg.norm(current_point - centroid) > tol:

    points = points.drop(neighbours.index.values)
    # print('removed points', neighbours.index.values)
    points.index = range(len(points.index))

    # print(points)
    # print('adding center')
    # print(current_point)
    centers = centers.append(current_point)
    # print(centers)

    # Initialize a set of non-clustered points U := X and the set of clusters C := ∅.
    #   Length U != ∅:
    #   Choose a random point x0.
    #   While the process is not stable
    #       Form a cluster c = {x ∈ X|ρ(x, x0) < R}.
    #       x0 := 1/|c| Ex∈c x.
    #   U := U \ c, C := C ∪ {c}
    #   Return the set of received clusters C



# print(centers)
clusters = []
# print('clusters')
# print('clusters')
# print('clusters')
centers.index = range(len(centers.index))
# print(centers)

for index in range(len(centers)):
    # print('center index', index)
    points = allPoints
    center = centers.loc[[index]].values[0]
    current_point = pd.DataFrame([center], columns=valueColumns)

    # print(center)

    current_point = pd.DataFrame(current_point, columns=valueColumns)

    pointsWithCenter = points.append(current_point, ignore_index = True)
    current_point_index = len(pointsWithCenter) - 1
    # print(pointsWithCenter)
    
    distances = pd.DataFrame(cdist(pointsWithCenter, pointsWithCenter, 'euclidean'))
    distancesToPoint = distances[current_point_index]
    neighboursIndexes = [index for index, distance in enumerate(distancesToPoint) if distance < radius]

    neighbours = df.loc[neighboursIndexes[:-1]]
    # print(neighbours)
    print(neighbours.pacientas.values)
    clusters.append(neighbours.pacientas.values)

# print(clusters)

----
[0.0, 0.0, 1.0, 0.0, 0.0, 0.3333333333333333, 0.3333333333333333, 0.0, 0.6666666666666666, 0.0]
    p1   p2   p3   p4   p5        p6        p7   p8        p9  p10
0  0.0  0.0  1.0  0.0  0.0  0.333333  0.333333  0.0  0.666667  0.0
     p1   p2   p3   p4   p5        p6        p7   p8        p9  p10
0   1.0  1.0  1.0  1.0  1.0  1.000000  1.000000  0.0  0.000000  1.0
1   1.0  1.0  1.0  1.0  1.0  1.000000  1.000000  0.0  0.000000  0.0
2   1.0  0.0  1.0  0.0  0.0  0.000000  1.000000  0.0  0.000000  1.0
3   1.0  1.0  0.0  0.0  1.0  0.000000  1.000000  0.0  0.000000  0.0
4   0.0  0.0  1.0  0.0  0.0  0.000000  0.000000  0.0  1.000000  0.0
5   0.0  1.0  1.0  1.0  1.0  0.000000  1.000000  0.0  0.000000  0.0
6   0.0  0.0  1.0  0.0  0.0  1.000000  1.000000  0.0  0.000000  0.0
7   1.0  1.0  1.0  0.0  1.0  0.000000  1.000000  0.0  0.000000  1.0
8   0.0  0.0  1.0  0.0  0.0  0.000000  1.000000  1.0  0.000000  1.0
9   0.0  0.0  1.0  0.0  0.0  0.000000  0.000000  0.0  1.000000  0.0
10  1.0  1.0  1.0

KeyError: ignored